# 튜토리얼 파트 5 번외
* tf-idf를 통해 벡터화 해보고 xgboost를 사용해본다.

In [1]:
import pandas as pd
"""
header = 0 은 파일의 첫 번째 줄에 열 이름이 있음을 나타내며 
delimiter = \t 는 필드가 탭으로 구분되는 것을 의미한다.
quoting = 3은 쌍따옴표를 무시하도록 한다.
"""
# QUOTE_MINIMAL (0), QUOTE_ALL (1), 
# QUOTE_NONNUMERIC (2) or QUOTE_NONE (3).

# 레이블인 sentiment 가 있는 학습 데이터
train = pd.read_csv('data/labeledTrainData.tsv', 
                    header=0, delimiter='\t', quoting=3)
# 레이블이 없는 테스트 데이터
test = pd.read_csv('data/testData.tsv', 
                   header=0, delimiter='\t', quoting=3)
print(train.shape)
print(test.shape)

(25000, 3)
(25000, 2)


In [2]:
# 긍정과 부정 리뷰가 어떻게 들어있는지 카운트 해본다.
# 긍정과 부정리뷰가 각각 동일하게 12,500개씩 들어있다.
train['sentiment'].value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

In [3]:
# 학습데이터에는 sentiment 데이터가 있다.
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
id           25000 non-null object
sentiment    25000 non-null int64
review       25000 non-null object
dtypes: int64(1), object(2)
memory usage: 586.0+ KB


In [4]:
# 테스트 데이터에는 sentiment가 없으며 이 sentiment를 예측하는 게 이 경진대회의 미션이다.
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
id        25000 non-null object
review    25000 non-null object
dtypes: object(2)
memory usage: 390.7+ KB


In [5]:
# 데이터 전처리는 튜토리얼 파트1~4까지 공통으로 사용되기 때문에 별도의 파이썬 파일로 분리했다.
# 그리고 캐글에 있는 코드를 병렬처리하도록 멀티프로세싱 코드를 추가했다.
from KaggleWord2VecUtility import KaggleWord2VecUtility

In [6]:
# 학습데이터를 전처리 한다.
%time train['review_clean'] = KaggleWord2VecUtility.apply_by_multiprocessing(\
    train['review'], KaggleWord2VecUtility.review_to_join_words, workers=4)

CPU times: user 129 ms, sys: 142 ms, total: 271 ms
Wall time: 1min


In [7]:
# 학습데이터와 동일하게 테스트 데이터에 대해서도 전처리 한다.
%time test['review_clean'] = KaggleWord2VecUtility.apply_by_multiprocessing(\
    test['review'], KaggleWord2VecUtility.review_to_join_words, workers=4)

CPU times: user 143 ms, sys: 193 ms, total: 335 ms
Wall time: 1min 4s


In [8]:
# 전처리한 학습 데이터 10개만을 불러와서 본다.
train['review_clean'][:10]

0    with all this stuff go down at the moment with...
1    the classic war of the world by timothi hine i...
2    the film start with a manag nichola bell give ...
3    it must be assum that those who prais this fil...
4    superbl trashi and wondrous unpretenti s explo...
5    i dont know whi peopl think this is such a bad...
6    this movi could have been veri good but come u...
7    i watch this video at a friend s hous i m glad...
8    a friend of mine bought this film for and even...
9    this movi is full of refer like mad max ii the...
Name: review_clean, dtype: object

In [9]:
# 전처리한 테스트 데이터 10개만을 불러와서 본다.
test['review_clean'][:10]

0    natur in a film who s main theme are of mortal...
1    this movi is a disast within a disast film it ...
2    all in all this is a movi for kid we saw it to...
3    afraid of the dark left me with the impress th...
4    a veri accur depict of small time mob life fil...
5    as valuabl as king tut s tomb ok mayb not that...
6    this has to be one of the biggest misfir ever ...
7    this is one of those movi i watch and wonder w...
8    the worst movi i ve seen in year and i ve seen...
9    five medic student kevin bacon david labraccio...
Name: review_clean, dtype: object

In [10]:
# X_train과 X_test에 리뷰 데이터를 담아주고 이 데이터를 TF-IDF를 통해 임베딩(벡터화)해본다. 
X_train = train['review_clean']
X_test = test['review_clean']

## TF-IDF
TF(단어 빈도, term frequency)는 특정한 단어가 문서 내에 얼마나 자주 등장하는지를 나타내는 값으로, 이 값이 높을수록 문서에서 중요하다고 생각할 수 있다. 하지만 단어 자체가 문서군 내에서 자주 사용되는 경우, 이것은 그 단어가 흔하게 등장한다는 것을 의미한다. 이것을 DF(문서 빈도, document frequency)라고 하며, 이 값의 역수를 IDF(역문서 빈도, inverse document frequency)라고 한다. TF-IDF는 TF와 IDF를 곱한 값이다.

IDF 값은 문서군의 성격에 따라 결정된다. 예를 들어 '원자'라는 낱말은 일반적인 문서들 사이에서는 잘 나오지 않기 때문에 IDF 값이 높아지고 문서의 핵심어가 될 수 있지만, 원자에 대한 문서를 모아놓은 문서군의 경우 이 낱말은 상투어가 되어 각 문서들을 세분화하여 구분할 수 있는 다른 낱말들이 높은 가중치를 얻게 된다.

역문서 빈도(IDF)는 한 단어가 문서 집합 전체에서 얼마나 공통적으로 나타나는지를 나타내는 값이다. 전체 문서의 수를 해당 단어를 포함한 문서의 수로 나눈 뒤 로그를 취하여 얻을 수 있다.

* 출처 : [TF-IDF - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/TF-IDF)

\begin{equation*}
\text{tfidf}(w, d) = \text{tf} \times (\log\big(\frac{N + 1}{N_w + 1}\big) + 1)
\end{equation*}


* 싸이킷런 공식문서 : [4.2. Feature extraction — scikit-learn 0.19.1 documentation](http://scikit-learn.org/stable/modules/feature_extraction.html)

In [11]:
from sklearn.pipeline import Pipeline

### TfidfVectorizer()

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

char_vectorizer = TfidfVectorizer(analyzer='char', max_features=10000, ngram_range=(1, 9))
char_vectorizer

TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 9), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [13]:
char_vectorizer.fit(X_train)

TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 9), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [14]:
X_train_char = char_vectorizer.transform(X_train)
X_train_char

<25000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 50211683 stored elements in Compressed Sparse Row format>

In [15]:
word_vectorizer = TfidfVectorizer(
    analyzer='word', 
    max_features=30000, 
    ngram_range=(1, 2))
word_vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=30000, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [16]:
word_vectorizer.fit(X_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=30000, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [17]:
X_train_word = word_vectorizer.transform(X_train)
X_train_word

<25000x30000 sparse matrix of type '<class 'numpy.float64'>'
	with 5837126 stored elements in Compressed Sparse Row format>

In [18]:
X_test_char = char_vectorizer.transform(X_test)
X_test_char

<25000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 49444189 stored elements in Compressed Sparse Row format>

In [19]:
X_test_word = word_vectorizer.transform(X_test)
X_test_word

<25000x30000 sparse matrix of type '<class 'numpy.float64'>'
	with 5681296 stored elements in Compressed Sparse Row format>

In [20]:
from scipy.sparse import hstack

X_train = hstack([X_train_char, X_train_word])
X_train

<25000x40000 sparse matrix of type '<class 'numpy.float64'>'
	with 56048809 stored elements in COOrdinate format>

In [21]:
X_test = hstack([X_test_char, X_test_word])
X_test

<25000x40000 sparse matrix of type '<class 'numpy.float64'>'
	with 55125485 stored elements in COOrdinate format>

In [22]:
y_train = train["sentiment"]

print(y_train.shape)
y_train.head()

(25000,)


0    1
1    1
2    0
3    0
4    1
Name: sentiment, dtype: int64

In [23]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤포레스트 분류기를 사용
forest = RandomForestClassifier(
    n_estimators = 100, n_jobs = -1, random_state=2018)
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=2018, verbose=0,
            warm_start=False)

In [24]:
%time forest = forest.fit(X_train, train['sentiment'])

CPU times: user 5min 32s, sys: 3.71 s, total: 5min 36s
Wall time: 2min 9s


In [25]:
%time result = forest.predict(X_test)

CPU times: user 37.2 s, sys: 608 ms, total: 37.8 s
Wall time: 15.2 s


In [26]:
output = pd.DataFrame(data={'id':test['id'], 'sentiment':result})
output.head()

,id,sentiment
0,"""12311_10""",1
1,"""8348_2""",0
2,"""5828_4""",1
3,"""7186_2""",0
4,"""12128_7""",1


In [27]:
output_sentiment = output['sentiment'].value_counts()
print(output_sentiment[0] - output_sentiment[1])
output_sentiment

-244


1    12622
0    12378
Name: sentiment, dtype: int64

In [28]:
output.to_csv('data/tutorial_4_tfidf_char_word.csv', index=False, quoting=3)

In [29]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y_train)

params = {
    'booster': 'gblinear',
    'objective': 'multi:softmax',
    'eval_metric': 'merror',
    'lambda': 2.0,
    'alpha': 1.0,
    'lambda_bias': 6.0,
    'num_class': 5,
    'nthread': 8,
    'silent': 1,
}

%time booster = xgb.train(params, dtrain, num_boost_round=90)

CPU times: user 6min 30s, sys: 2.73 s, total: 6min 33s
Wall time: 3min 6s


In [30]:
dtest = xgb.DMatrix(X_test.toarray())

predictions = booster.predict(dtest)

print(predictions.shape)
predictions[0:10]

(25000,)


array([1., 0., 1., 0., 1., 1., 0., 1., 0., 0.], dtype=float32)

In [38]:
output = pd.DataFrame(data={'id':test['id'], 'sentiment':result})
output.head()

,id,sentiment
0,"""12311_10""",1
1,"""8348_2""",0
2,"""5828_4""",1
3,"""7186_2""",0
4,"""12128_7""",1


In [39]:
output_sentiment = output['sentiment'].value_counts()
print(output_sentiment[0] - output_sentiment[1])
output_sentiment

-244


1    12622
0    12378
Name: sentiment, dtype: int64

In [36]:
submission.to_csv("data/tutorial_4_tfidf_xgboost_char_word.csv")

In [37]:
# 캐글 스코어 0.87564
270/578

0.4671280276816609